# Choosing Subsets with Maximum Weighted Average
### (Newton Iteration Method)
https://www.ics.uci.edu/~eppstein/pubs/EppHir-TR-95-12.pdf

### Let $S$ be the set of grade score values and weights of each lesson.
### $S:\{(v_1,w_1),(v_1,w_1),...,(v_n,w_n)\}$  
### $v_i$ score value of lesson $i$
### $w_i$ score weight of lesson $i$

### Let $n$ be the number of elements in $S$, such that $|S| = n$, and $k$ be the number of elements to exclude from the chosen subset.
Suppose that some $(n − k)$-element set $T ⊂ S$ has weighted average at least $A$.  
We can write this as an inequality of the form:
# $A \leq A(T) = \frac{\sum_{i\in T}v_i}{\sum_{i\in T}w_i}$
Assume that all weights are positive, can rewrite as:  
# $\sum\limits_{i\in T}(v_i-Aw_i)\geq 0$

ie. If some $T$ has a weighted average greater than or equal to some value $A$, then $\sum\limits_{i\in T}(v_i-Aw_i)\geq 0$

For each lesson score $i$, define the decreasing linear function:
### $f_i(A)=v_i - Aw_i$
then also define:
### $F(A) = \max\limits_{|T|=n-k}\sum\limits_{i\in T}f_i(A)$
Which can be computed by selecting the $n-k$ largest values of $f_i(A)$ from above
### $F(A)$ is a piecewise linear decreasing function since it is the maximum of ${n \choose n-k}$ decreasing linear functions. Thus we can simply find the root of F(A) using Newton iteration method.

In [92]:
import numpy as np
import time

In [289]:
real_scores = [4.0,1.0,1.0,0.0,1.0,3.0,2.0,2.0,3.0,2.0,2.0,4.0,1.0,1.0]
max_scores =  [4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0]
weights =     [5.0,45.0,45.0,5.0,30.0,20.0,15.0,30.0,5.0,45.0,45.0,15.0,30.0,5.0]

score_weights = [(weights[i]*real_scores[i]/max_scores[i],weights[i]) for i in range(len(real_scores))]
score_weights

[(5.0, 5.0),
 (11.25, 45.0),
 (11.25, 45.0),
 (0.0, 5.0),
 (7.5, 30.0),
 (15.0, 20.0),
 (7.5, 15.0),
 (15.0, 30.0),
 (3.75, 5.0),
 (22.5, 45.0),
 (22.5, 45.0),
 (15.0, 15.0),
 (7.5, 30.0),
 (1.25, 5.0)]

In [290]:
norm_score = sum([score_weights[i][0] for i in range(len(score_weights))])
norm_max_score = sum([score_weights[i][1] for i in range(len(score_weights))])
print("Final Score (#/#) :", norm_score,"/",norm_max_score)

percentage_score = norm_score/norm_max_score
print("Final Score (%)   :",percentage_score)

Final Score (#/#) : 145.0 / 340.0
Final Score (%)   : 0.4264705882352941


In [303]:
# choose = (n-k)
n = len(real_scores)
k = 11
choose = n-k

# Calculated Weighted Average of A Subset
def A(set_T):
    scores = [score_weights[i][0] for i in set_T]
    weights = [score_weights[i][1] for i in set_T]
    
    # Return weighted average
    if sum(weights) != 0:
        weighted_avg = sum(scores)/sum(weights)
        return weighted_avg
    
    # If set_T is an empty or unweighted set of scores
    else:
        return 0

# Get f_i scores for a given (current iteration's) weighted average
def get_fi_a(a):
    
    # f_i(a) = v_i - a*w_i
    return np.asarray([sw[0] for sw in score_weights]) - a*np.asarray([sw[1] for sw in score_weights])

# Get Subset which maximizes improvement over current weighted average
def F(a):
    
    # Get f_i scores for a given (current iteration's) weighted average, a
    fi_a = get_fi_a(a)
    
    # Get (n-k) highest f_i scores
    best_set = np.argpartition(fi_a,-choose)[-choose:]
    
    return best_set

In [405]:
# T_i = T_0
T = np.random.choice(np.asarray((range(n))), replace=False, size=choose)

# Weighted Avg of Initial Subset
print("\nT_0",T)
print("Weighted Avg of T_0:", A(T))

# T_i_plus_1
T_prev = np.asarray([])
i=0
while A(T) != A(T_prev):
    T_prev = T    
    T = F(A(T_prev))
    print("\nT_"+str(i+1),T)
    print("Weighted Avg of T_"+str(i+1)+":", A(T))
    i+=1


T_0 [9 6 4]
Weighted Avg of T_0: 0.4166666666666667

T_1 [ 9  5 11]
Weighted Avg of T_1: 0.65625

T_2 [ 0  5 11]
Weighted Avg of T_2: 0.875

T_3 [ 8  0 11]
Weighted Avg of T_3: 0.95

T_4 [ 8  0 11]
Weighted Avg of T_4: 0.95
